In [47]:
#importation des packages
import pandas as pd
import numpy as np
import re

In [48]:
#importation des données du baromètre
df=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/donnees_brutes_2021.csv", sep=";")

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,3,68,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
#variable géographique harmonisée
p=re.compile('(\D*) \(([0-9]{2,2})[0-9]{3,3}\)')
def ville_dep(x):
    m=p.match(x)
    return m.group(1)+' ' +m.group(2)
df['Ville']=df['Commune'].apply(ville_dep)

In [50]:
#on met les indicateurs en numérique
var_indicateur=['Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global', 'Indicateur_global_declare']
def yo_float(x):
    return float(str(x).replace(',','.'))

def indicateurs_nombres(df1, liste_indicateur=var_indicateur):
    for indicateur in var_indicateur :
        if indicateur in df1.columns:
            df1[indicateur]=df1[indicateur].apply(yo_float)
    return df1
df=indicateurs_nombres(df)


In [51]:
#création d'un indicateur de marchabilité alternatif
liste_var=['Deplacement_a_pied',  'Agencement_voirie', 'conflits_pietons', 'Conducteurs_respectent', 'Circulation_vehicules_motorises',  
 'Marche_securite',  'Securite_rues_services', 'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines', 
 'Acces_pietons', 'Traverse_carrefour', 'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole', 
 'Pistes_cyclables', 'Espaces_pietons', 'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus', 
 'Signaletique_specifique','Amenagements_confort_marcheurs', 'Trottoirs_eclaires', 
 'Stationnement_espaces_pietons', 'Signaletique', 'Resume_condition_marche'] 
df['somme_note']=0
df['nombre_notes_donnees']=0
def to_0(x):
    if x==np.nan:
        return 0
    else : 
        return x
def to_1(x):
    if x==np.nan:
        return 0
    else :
        return 1
for i in liste_var :
    df['somme_note']+=df[i].apply(to_0)
    df['nombre_notes_donnees']+=df[i].apply(to_1)
df['note_marchabilite']=df['somme_note']/df['nombre_notes_donnees']


In [52]:
comp=(df['note_marchabilite']-df['Indicateur_global']).dropna()
sum(abs(comp))/len(comp)

0.18545771134657796

In [53]:
#fonction qui crée des moyennes par ville
df2=df.copy()
def tt_dataville_(df, var_note, commune='Ville') :
    liste_ville=df[commune].unique()
    grosse_ville=[]
    for ville in liste_ville :
        df_ville1=df[df[commune]==ville]
        if df_ville1[commune].count()>=40 :
            grosse_ville+=[ville]#on va se concentrer sur les grandes villes
    print(len(grosse_ville))
    dataville=pd.DataFrame()
    dataville[commune]=np.array(grosse_ville)
    for col in var_note:
        liste=[]
        for ville in grosse_ville:
            donnees=df[df[commune]==ville]
            moyenne=np.mean(donnees[col])
            liste.append(moyenne)
        dataville['moyenne'+col]=np.array(liste)
    return dataville
dataville_indic=tt_dataville_(df2, var_note=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche'])
dataville_indic
    

199


,Ville,moyenneIndicateur_global,moyenneIndicateur_securite,moyenneIndicateur_confort,moyenneIndicateur_ressenti_global,moyenneIndicateur_effort_ville,moyenneIndicateur_amenagements_services,moyenneResume_condition_marche
0,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857
1,Aix-en-Provence 13,3.253586,3.543715,3.044108,3.380184,2.637097,3.662826,3.479263
2,Aix-les-Bains 73,3.903217,4.012607,3.894505,3.807692,3.619231,4.182051,4.138462
3,Ajaccio 20,2.453236,2.780708,2.247706,2.698777,2.011468,2.527523,2.403670
4,Albi 81,3.486289,3.736111,3.236152,3.659864,3.010204,3.789116,3.632653
...,...,...,...,...,...,...,...,...
194,Vincennes 94,4.025573,4.402778,4.047619,4.040123,3.662037,3.975309,4.407407
195,Vitré 35,3.320939,3.299964,3.333952,3.471861,2.743506,3.755411,3.467532
196,Vitrolles 13,3.322451,3.452465,3.287726,3.380282,2.982394,3.509390,3.464789
197,Vitry-sur-Seine 94,2.673956,3.226608,2.469925,2.877193,1.973684,2.822368,2.394737


In [54]:
df

,Commune,Code_INSEE,Commune_cp,Dep,Cp,Nom_dep,Nom_region,Malvoyant,Taille_commune_2,Pratique de la marche,...,Indicateur_securite,Indicateur_confort,Indicateur_effort_ville,Indicateur_amenagements_services,Indicateur_global,Indicateur_global_declare,Ville,somme_note,nombre_notes_donnees,note_marchabilite
0,Acigné (35690),35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne,Non,De 5000 et 19 999 habitants,1 à 3 fois par semaine/Presque jamais,...,3.444444,3.428571,3.00,2.666667,3.141270,3.0,Acigné 35,74.0,23,3.217391
1,Acigné (35690),35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne,Non,De 5000 et 19 999 habitants,Tous les jours ou presque/1 à 3 fois par mois,...,5.111111,4.428571,5.00,4.000000,4.641270,5.0,Acigné 35,111.0,23,4.826087
2,Acigné (35690),35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne,Non,De 5000 et 19 999 habitants,1 à 3 fois par mois/1 à 3 fois par mois,...,4.111111,4.000000,3.25,3.666667,3.772222,4.0,Acigné 35,93.0,23,4.043478
3,Acigné (35690),35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne,Non,De 5000 et 19 999 habitants,Tous les jours ou presque/1 à 3 fois par semaine,...,5.000000,4.857143,5.50,6.000000,5.371429,6.0,Acigné 35,121.0,23,5.260870
4,Acigné (35690),35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne,Non,De 5000 et 19 999 habitants,Tous les jours ou presque/1 à 3 fois par mois,...,4.625000,3.714286,4.00,4.333333,4.267857,4.0,Acigné 35,NaN,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43262,Châtillon (69380),69050,Châtillon,69,69380,Rhône,Auvergne-Rhône-Alpes,Non,Moins de 5000 habitants,Tous les jours ou presque/Tous les jours ou pr...,...,3.555556,3.285714,3.50,2.000000,3.368254,4.0,Châtillon 69,83.0,23,3.608696
43263,Châtillon (69380),69050,Châtillon,69,69380,Rhône,Auvergne-Rhône-Alpes,Non,Moins de 5000 habitants,1 à 3 fois par semaine/1 à 3 fois par semaine,...,3.222222,3.000000,3.00,3.333333,3.111111,4.0,Châtillon 69,72.0,23,3.130435
43264,Massy (76270),76415,Massy,76,76270,Seine-Maritime,Normandie,Non,Moins de 5000 habitants,Presque jamais/Presque jamais,...,NaN,NaN,NaN,NaN,NaN,NaN,Massy 76,NaN,23,NaN
43265,Montreuil (62170),62588,Montreuil,62,62170,Pas-de-Calais,Hauts-de-France,Non,Moins de 5000 habitants,Tous les jours ou presque/1 à 3 fois par semaine,...,5.333333,4.285714,4.00,4.500000,4.790476,6.0,Montreuil 62,116.0,23,5.043478


In [55]:
df_geo=df[['Code_INSEE', 'Commune_cp', 'Dep', 'Cp', 'Nom_dep', 'Nom_region', 'Ville']]
dataville_indic=dataville_indic.merge(df_geo, how='left')
dataville_indic

,Ville,moyenneIndicateur_global,moyenneIndicateur_securite,moyenneIndicateur_confort,moyenneIndicateur_ressenti_global,moyenneIndicateur_effort_ville,moyenneIndicateur_amenagements_services,moyenneResume_condition_marche,Code_INSEE,Commune_cp,Dep,Cp,Nom_dep,Nom_region
0,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne
1,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne
2,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne
3,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne
4,Acigné 35,4.726466,4.892290,4.559767,4.891156,4.571429,4.717687,5.142857,35001,Acigné,35,35690,Ille-et-Vilaine,Bretagne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26623,Voreppe 38,3.310083,3.223639,3.212828,3.309524,2.943878,3.860544,3.204082,38565,Voreppe,38,38340,Isère,Auvergne-Rhône-Alpes
26624,Voreppe 38,3.310083,3.223639,3.212828,3.309524,2.943878,3.860544,3.204082,38565,Voreppe,38,38340,Isère,Auvergne-Rhône-Alpes
26625,Voreppe 38,3.310083,3.223639,3.212828,3.309524,2.943878,3.860544,3.204082,38565,Voreppe,38,38340,Isère,Auvergne-Rhône-Alpes
26626,Voreppe 38,3.310083,3.223639,3.212828,3.309524,2.943878,3.860544,3.204082,38565,Voreppe,38,38340,Isère,Auvergne-Rhône-Alpes


In [56]:
#fonction qui attribue à chaque note sur 6 sa lettre selon la classification FUB
def to_lettre(x) :
    x=float(x)
    if x>4.6 :
        return 'A+'
    elif x>4.3 :
        return 'A'
    elif x>3.9 :
        return 'B'
    elif x>3.5 :
        return 'C'
    elif x>3.1 :
        return 'D'
    elif x>2.7 :
        return 'E'
    elif x>2.3 :
        return 'F'
    elif x>=0 :
        return 'G'
    else :
        return np.nan
    
def lettre(df):
    df['lettre_FUB']=df['moyenneIndicateur_global'].apply(to_lettre)
    return df

In [57]:
#on attribue cette lettre à toutes les notes
var_note=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche']
for indic in var_note :
    dataville_indic['lettre_FUB'+indic]=dataville_indic['moyenne'+indic].apply(to_lettre)

In [58]:
df_nom_comm=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/population par commune.csv", sep=";", skiprows=10)

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [59]:
#on fusionne avec la population totale
def prendre1(x) :
    return str(x)[0]
def prendre2(x):
    return(str(x)[1])
df_nom_comm['Ville']=df_nom_comm['LIBGEO']+' '+df_nom_comm['CODGEO'].apply(prendre1)+df_nom_comm['CODGEO'].apply(prendre2)
df_population=df_nom_comm[['Ville','Population totale']]

In [60]:
#df_accident=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/accidents.csv", sep=";")

#df_accident.merge(df_nom_comm, on=['CODGEO'])

In [61]:
listecol=['Indicateur_global', 'Indicateur_securite', 'Indicateur_confort', 'Indicateur_ressenti_global', 'Indicateur_effort_ville','Indicateur_amenagements_services', 'Resume_condition_marche']
liste_col=[]
liste_col_moy=[]
for i in listecol :
    liste_col.append('lettre_FUB'+i)
    liste_col.append('moyenne'+i)
    liste_col_moy.append('moyenne'+i)
for geo in [i for i in df_geo.columns] :
    liste_col.append(geo)

In [62]:
#on rajoute le revenu médian et le taux d'accidentalité pour ces villes
dataexp=dataville_indic[liste_col]
taux_pauvrete=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\taux_pauvrete.csv', sep=';')
taux_pauvrete_lecture=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\meta_cc_filosofi_2018_COM-geo2021.csv', sep=';')
taux_pauvrete_lecture['CODGEO']=taux_pauvrete_lecture['COD_MOD'].apply(str)#attention, certains sont pris pour des float et d'autres des strings !
taux_pauvrete['Taux_pauvrete']=taux_pauvrete['TP6018']
taux_pauvrete=taux_pauvrete[taux_pauvrete['TP6018'].notna()]
taux_pauvrete['niveau_vie_median_milliers']=taux_pauvrete['MED18']/1000
taux_pauvrete=taux_pauvrete[['CODGEO','Taux_pauvrete','niveau_vie_median_milliers']]
taux_pauvrete['CODGEO']=taux_pauvrete['CODGEO'].apply(str)
df_pauvrete=taux_pauvrete.merge(taux_pauvrete_lecture[['CODGEO','LIB_MOD']])
df_pauvrete['Ville1']=df_pauvrete['LIB_MOD']

p=re.compile('(\D*) [0-9]{2,2}')
def yo_ville1(x):
    m=p.match(x)
    return m.group(1)
dataexp['Ville1']=dataexp['Ville'].apply(yo_ville1)

df_accident=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/accidents.csv", sep=";")
df_accident=df_accident.rename(columns={'Ville':'Ville2'})
def moins_le (x):
    x=str(x)
    if x.split()[0]=='Le' or x.split()[0]=='La' :
        return x[3:]
    else :
        return x
dataexp['Ville2']=dataexp['Ville'].apply(moins_le)
dataexp=dataexp.merge(df_accident, how='left')
dataexp["nombre_accidents_pietons"] = dataexp["nombre d'accidents pietons"].fillna(0)

<ipython-input-62-a3bb9c831c3b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataexp['Ville1']=dataexp['Ville'].apply(yo_ville1)
<ipython-input-62-a3bb9c831c3b>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataexp['Ville2']=dataexp['Ville'].apply(moins_le)


In [63]:
#on fusionne le nombre d'habitants en prenant en compte la Corse
df_nom_comm=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/population par commune.csv", sep=";", skiprows=10)

def prendre1(x) :
    if len(str(x))==4 :
        return(str(0))    
    else:
        return str(x)[0]
def prendre2(x):
    if len(str(x))==4 :
        return str(x)[0]
    elif str(x)[1]=='A' or str(x)[1]=='B':
        return str(0)
    else :
        return str(x)[1]
    
df_nom_comm['Ville2']=df_nom_comm['LIBGEO']+' '+df_nom_comm['CODGEO'].apply(prendre1)+df_nom_comm['CODGEO'].apply(prendre2)
df_population=df_nom_comm[['Ville2','Population totale']]

In [64]:
#on met les notes sur 20 et on les adapte à une bonne présentation
df_population['Ville2']=df_population['Ville2'].apply(moins_le)
dataexp=dataexp.merge(df_population, how='left')
dataexp["nombre_accidents_pietons"] = dataexp["nombre d'accidents pietons"].fillna(0)
dataexp['accident pour 1000 habitants 2015-2019']=(dataexp["nombre_accidents_pietons"]*1000)/dataexp['Population totale']
dataexp['plus_accident']=(dataexp['accident pour 1000 habitants 2015-2019']>42784/67000).apply(float)

for i in liste_col_moy :
    dataexp[i]=(dataexp[i]-1)*4

dataexp=dataexp.merge(df_pauvrete[['Ville1', 'niveau_vie_median_milliers']], how='left')
dataexp["revenu médian en milliers d'euros"]=dataexp['niveau_vie_median_milliers']
dataexp=dataexp[['Population totale', 'accident pour 1000 habitants 2015-2019',"revenu médian en milliers d'euros"]+liste_col]
def round2 (x):
    return round(x,2)
#for i in ['accident pour 1000 habitants 2015-2019', 'note'] :
    #dataexp[i]=dataexp[i].apply(round2)
#dataexp.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes_villes50.csv', index=False, sep=';')

<ipython-input-64-9c0b145e45e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population['Ville2']=df_population['Ville2'].apply(moins_le)


In [68]:
def compte(x) :
    return sum(df['Ville']==x)
dataexp['nombre répondants']=dataexp['Ville'].apply(compte)
def inv (x):
    return 1/np.sqrt(x)
def ecart(x) :
    return np.std(df[df['Ville']==x]['Indicateur_global'])*4
dataexp['écart-type']=dataexp['Ville'].apply(ecart).round(2)
dataexp['intervalle de confiance']=(dataexp['moyenneIndicateur_global']-(dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv))).round(2).apply(str)+' ; '+(dataexp['moyenneIndicateur_global']+(dataexp['écart-type']*1.96*dataexp['nombre répondants'].apply(inv))).round(2).apply(str)
def to_march(x):
    villmarch=df[df['Ville']==x]
    nbremarch=sum(villmarch['Membre_association_marche']=='Oui')
    return nbremarch/(nbremarch+sum(villmarch['Membre_association_marche']=='Non'))
dataexp["pourcentage de membres d'associations de marche"]=dataexp['Ville'].apply(to_march).round(2)
dataexport=dataexp[['Ville', 'Population totale',
       'accident pour 1000 habitants 2015-2019',
       "revenu médian en milliers d'euros", 'nombre répondants', 'écart-type',
       'intervalle de confiance', "pourcentage de membres d'associations de marche"]+liste_col]

In [ ]:
dataexport

In [19]:
dataexport.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes_villes50.csv', index=False, sep=';')

In [8]:
df3=df.copy()
def valeur_taille(df1):
    liste=[]
    for i in df1['Taille_commune_2']:
        if i=='De 5000 et 19 999 habitants':
            liste.append('5 M-20 M habitants')
        elif i =='De 100 000 et 199 999 habitants':
            liste.append('100 M-200 M habitants')
        elif i =='De 20 000 et 49 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='De 50 000 et 99 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='200 000 habitants ou plus':
            liste.append('200 M habitants')
        elif i =='Moins de 5000 habitants':
            liste.append('-5 M habitants')
        else :
            liste.append(np.nan)
    df1['Taille']=np.array(liste)
valeur_taille(df3)

In [9]:

df3['lettre']=df['Indicateur_global'].apply(to_lettre)
df3['lettre_securite']=df['Indicateur_securite'].apply(to_lettre)
df3['lettre_confort']=df['Indicateur_confort'].apply(to_lettre)
for lettree in df3['lettre'].unique():
    print(lettree, sum(df3['lettre']==lettree))
for taille in df3['Taille'].unique() :
    print(taille, to_lettre(np.mean(df3[df3['Taille']==taille]['Indicateur_global'])), 4*(np.mean(df3[df3['Taille']==taille]['Indicateur_global'])-1))
    

D 6642
A+ 3930
C 6093
B 4706
A 2751
E 6491
F 5485
G 6460
nan 0
5 M-20 M habitants D 9.714369956996881
100 M-200 M habitants D 9.463923241695769
20 M-100 M habitants D 9.017220604793577
200 M habitants E 8.229114875187053
-5 M habitants D 9.964667363457243


In [10]:
df_cycl=pd.read_csv(r'C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/barometre cyclable/Base2019_communes_selectionnes.csv', sep=';')

In [11]:
df_cycl

,Nom_com_dep,Depcom,Classe,Dep_nom,Reg_nom,note_globale_moy,note_progres_moy,note_ressenti_general_moy,note_securite_moy,note_confort_moy,note_importance_velo_moy,note_stat_services_moy,n_tot_valide_net,classement_note_globale_classe,note_globale_moy_classe,Climat_Velo_num
0,Saint-Lunaire (35),35287,[0-20 000],Ille-et-Vilaine,Bretagne,"4,4695670995671","1,61038961038961","4,5","4,38095238095238","4,30909090909091","4,87987012987013","4,27792207792208",77,1,"2,76",7
1,Séné (56),56243,[0-20 000],Morbihan,Bretagne,"4,22766937669377","1,13934426229508","4,45663956639566","4,1219512195122","4,11869918699187","4,75813008130081","3,68292682926829",123,2,"2,76",6
2,Marseillan (34),34150,[0-20 000],Hérault,Occitanie,"4,19210752688172","1,09395973154362","4,36129032258065","4,04408602150538","4,04387096774194","4,63387096774194","3,87741935483871",155,3,"2,76",6
3,Sceaux (92),92071,[0-20 000],Hauts-de-Seine,Île-de-France,"3,97336134453782","0,563025210084034","4,02380952380952","3,87535014005602","3,97478991596639","4,22478991596639","3,76806722689076",119,4,"2,76",6
4,La Flèche (72),72154,[0-20 000],Sarthe,Pays de la Loire,"3,9406146572104","0,191489361702128","4,1193853427896","3,56382978723404","3,64964539007092","4,05673758865248","4,31347517730497",141,5,"2,76",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,Lille (59),59350,]200 000 et plus],Nord,Hauts-de-France,"3,03135260115607","0,370615744349182","3,10764290301863","2,73782915863841","3,00339113680154","2,77957610789981","3,52832369942197",2595,7,"3,04",3
766,Toulouse (31),31555,]200 000 et plus],Haute-Garonne,Occitanie,"2,88301993673214","0,114655364711131","2,89108364599426","2,50544397851836","3,08280732730082","2,46319796954315","3,47256676230413",4531,8,"3,04",3
767,Montpellier (34),34172,]200 000 et plus],Hérault,Occitanie,"2,48073120686439","-0,0871780356637419","2,65263010632345","2,23185972766275","2,22266368214885","2,14171796306659","3,15478455512031",3574,9,"3,04",2
768,Nice (06),06088,]200 000 et plus],Alpes-Maritimes,Provence-Alpes-Côte d'Azur,"2,36855324074074","0,158813263525305","2,31047453703704","2,1640625","2,19513888888889","2,13802083333333","3,03506944444444",576,10,"3,04",2


In [12]:
p=re.compile('(\D*) \((.{2,2})')
def ville_dep(x):
    m=p.match(x)
    a=m.group(2)
    if a=='2A'or a=='2B':
        a='20'
    return m.group(1)+' ' +a
df_cycl['Ville']=df_cycl['Nom_com_dep'].apply(ville_dep)
def yo_float(x):
    return float(str(x).replace(',','.'))

In [22]:
df_note=dataville_indic.merge(df_cycl, how='left', on = 'Ville')
note_cycl=['note_globale_moy','note_ressenti_general_moy','note_securite_moy','note_confort_moy']
for indic in note_cycl :
    df_note[indic]=df_note[indic].apply(yo_float)
    df_note['lettre_cycl'+indic]=df_note[indic].apply(to_lettre)
df_note

,Ville,moyenneIndicateur_global,moyenneIndicateur_securite,moyenneIndicateur_confort,moyenneIndicateur_ressenti_global,lettreIndicateur_global,lettreIndicateur_securite,lettreIndicateur_confort,lettreIndicateur_ressenti_global,Nom_com_dep,...,note_importance_velo_moy,note_stat_services_moy,n_tot_valide_net,classement_note_globale_classe,note_globale_moy_classe,Climat_Velo_num,lettre_cyclnote_globale_moy,lettre_cyclnote_ressenti_general_moy,lettre_cyclnote_securite_moy,lettre_cyclnote_confort_moy
0,Aix-en-Provence 13,3.253586,3.543715,3.044108,3.380184,D,C,E,D,Aix-en-Provence (13),...,"2,13352272727273","2,81363636363636",792.0,24.0,"2,79",2.0,F,F,G,F
1,Aix-les-Bains 73,3.903217,4.012607,3.894505,3.807692,B,B,C,C,Aix-les-Bains (73),...,"3,08684210526316","3,79578947368421",95.0,14.0,"2,68",4.0,D,D,E,C
2,Ajaccio 20,2.453236,2.780708,2.247706,2.698777,F,E,G,F,Ajaccio (2A),...,"1,82450980392157","3,09490196078431",255.0,67.0,"2,71",1.0,G,G,G,G
3,Albi 81,3.486289,3.736111,3.236152,3.659864,D,C,D,C,Albi (81),...,"3,07976190476191","3,38761904761905",210.0,29.0,"2,68",4.0,D,D,E,D
4,Alfortville 94,2.429223,2.801101,2.156334,2.742138,F,E,G,E,Alfortville (94),...,"2,26433121019108","3,03821656050955",157.0,143.0,"2,68",2.0,F,E,F,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,Villeurbanne 69,3.230656,3.367734,3.147941,3.088375,D,D,D,E,Villeurbanne (69),...,"2,75455005055612","3,4273003033367",989.0,10.0,"2,79",3.0,E,E,E,E
150,Vincennes 94,4.025573,4.402778,4.047619,4.040123,B,A,B,B,Vincennes (94),...,"3,12735849056604","3,26603773584906",212.0,25.0,"2,68",4.0,D,D,E,D
151,Vitré 35,3.320939,3.299964,3.333952,3.471861,D,D,D,D,Vitré (35),...,"2,36163522012579","3,26918238993711",159.0,212.0,"2,76",3.0,E,E,F,E
152,Vitrolles 13,3.322451,3.452465,3.287726,3.380282,D,D,D,D,Vitrolles (13),...,"2,74596774193548","2,36451612903226",62.0,135.0,"2,68",2.0,F,E,F,F


In [43]:
df_export=df_note[['Ville', 'lettreIndicateur_global', 'moyenneIndicateur_global']]
df_export.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\villes note.csv', index=False, sep=';')

In [44]:
df_export

,Ville,lettreIndicateur_global,moyenneIndicateur_global
0,Aix-en-Provence 13,D,3.253586
1,Aix-les-Bains 73,B,3.903217
2,Ajaccio 20,F,2.453236
3,Albi 81,D,3.486289
4,Alfortville 94,F,2.429223
...,...,...,...
149,Villeurbanne 69,D,3.230656
150,Vincennes 94,B,4.025573
151,Vitré 35,D,3.320939
152,Vitrolles 13,D,3.322451


In [14]:
for lettree in df_note['lettreIndicateur_global'].unique():
    print(lettree, sum(df_note['lettreIndicateur_global']==lettree))

D 64
B 10
F 9
E 31
C 36
G 2
A 2


In [15]:
df_note['lettreIndicateur_global'].unique()

array(['D', 'B', 'F', 'E', 'C', 'G', 'A'], dtype=object)

In [23]:
dic={}
note_cycl=['note_globale_moy','note_ressenti_general_moy','note_securite_moy','note_confort_moy']
var_note=['Indicateur_global', 'Indicateur_ressenti_global', 'Indicateur_securite', 'Indicateur_confort']
for i in range(len(note_cycl)) :
    dic[var_note[i]]=note_cycl[i]

In [26]:

def tableau_correspondance(indic):
    correspond=pd.DataFrame()
    lettres=['A+', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
    for note_marche in lettres :
        liste_note_corr=[]
        df_cette_note=df_note[df_note['lettre'+indic]==note_marche]
        for note_cycl in lettres:
            liste_note_corr.append(sum(df_cette_note['lettre_cycl'+dic[indic]]==note_cycl))
        correspond[note_marche]=np.array([i for i in liste_note_corr])
    correspond['note_cycl']=np.array(lettres)
    correspond.set_index('note_cycl', inplace=True, drop=True)
    liste_note_total=[]
    total=pd.DataFrame()
    for i in lettres :
        nbre=np.sum(correspond[i])
        liste_note_total.append(nbre)
        total[i]=np.array([nbre])
    total['note_cycl']='Total'
    total.set_index('note_cycl', inplace=True, drop=True)
    correspond=pd.concat([correspond, total])
    correspond['Total']=sum([correspond[i] for i in lettres])
    return correspond
tableau_correspondance('Indicateur_global')

,A+,A,B,C,D,E,F,G,Total
note_cycl,,,,,,,,,
A+,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0
B,0,1,0,3,0,0,0,0,4
C,0,0,3,2,2,1,0,0,8
D,0,0,5,13,13,2,0,0,33
E,0,0,2,7,21,8,0,0,38
F,0,0,0,4,17,13,4,0,38
G,0,0,0,1,4,5,5,2,17
Total,0,1,10,30,57,29,9,2,138


In [32]:
indic='Indicateur_global'
df_cette_note=df_note[df_note['lettre'+indic]=='C']
sum(df_cette_note['lettre_cycl'+dic[indic]]=='G')

1

In [31]:
tableau_correspondance('Indicateur_ressenti_global')

,A+,A,B,C,D,E,F,G,Total
note_cycl,,,,,,,,,
A+,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0
B,0,1,2,1,2,0,0,0,6
C,0,1,0,5,4,0,0,0,10
D,0,1,4,21,10,2,0,0,38
E,0,0,2,12,22,7,1,0,44
F,0,0,0,5,13,15,0,0,33
G,0,0,0,1,0,3,2,1,7
Total,0,3,8,45,51,27,3,1,138


In [28]:
tableau_correspondance('Indicateur_securite')

,A+,A,B,C,D,E,F,G,Total
note_cycl,,,,,,,,,
A+,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0
C,1,2,2,3,0,0,0,0,8
D,0,1,3,7,2,0,0,0,13
E,0,1,10,19,11,1,0,0,42
F,0,0,2,17,17,4,0,0,40
G,0,0,0,8,15,10,2,0,35
Total,1,4,17,54,45,15,2,0,138


In [51]:
tableau_correspondance('Indicateur_confort')

,A+,A,B,C,D,E,F,G,Total
note_cycl,,,,,,,,,
A+,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0
B,0,0,1,2,1,0,0,0,4
C,0,0,3,2,2,0,1,0,8
D,0,0,3,11,12,5,2,0,33
E,0,0,1,4,13,19,1,0,38
F,0,0,0,1,11,16,9,1,38
G,0,0,0,0,2,8,2,5,17
Total,0,0,8,20,41,48,15,6,138


In [72]:
liste_note_total=[]
total=pd.DataFrame()
for i in lettres :
    nbre=np.sum(correspond[i])
    liste_note_total.append(nbre)
    total[i]=np.array([nbre])
total['note_cycl']='Total'
total.set_index('note_cycl', inplace=True, drop=True)
correspond=pd.concat([correspond, total])
correspond['Total']=sum([correspond[i] for i in lettres])
correspond

,A+,A,B,C,D,E,F,G,Total
note_cycl,,,,,,,,,
A+,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0
B,0,1,0,3,0,0,0,0,4
C,1,0,3,2,2,1,0,0,9
D,0,2,7,13,15,4,0,0,41
E,0,0,2,11,25,11,0,0,49
F,0,0,0,6,20,18,5,0,49
G,0,0,0,1,6,6,6,2,21
Total,1,3,12,36,68,40,11,2,173


In [73]:
correspond.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\correspond.csv', index=False, sep=';')

In [74]:
df_note[df_note['lettre_cycl']=='G']

,Ville,moyenneIndicateur_global,lettre,Nom_com_dep,Depcom,Classe,Dep_nom,Reg_nom,note_globale_moy,note_progres_moy,note_ressenti_general_moy,note_securite_moy,note_confort_moy,note_importance_velo_moy,note_stat_services_moy,n_tot_valide_net,classement_note_globale_classe,note_globale_moy_classe,Climat_Velo_num,lettre_cycl
3,Ajaccio 20,2.453236,F,Ajaccio (2A),2A004,]50 000-100 000],Corse-du-Sud,Corse,2.039203,"-0,142857142857143","2,10392156862745","1,68169934640523","1,49098039215686","1,82450980392157","3,09490196078431",255.0,67.0,"2,71",1.0,G
10,Antibes 06,2.945668,E,Antibes (06),06004,]50 000-100 000],Alpes-Maritimes,Provence-Alpes-Côte d'Azur,2.237419,"0,0194647201946472","2,2831715210356","2,05622977346278","2,05097087378641","1,91808252427184","2,87864077669903",412.0,61.0,"2,71",1.0,G
15,Aubagne 13,3.266075,D,Aubagne (13),13005,[20 000-50 000],Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,2.076970,"-0,218487394957983","2,35537190082645","1,98898071625344","1,74380165289256","1,78925619834711","2,50743801652893",121.0,213.0,"2,68",1.0,G
17,Aubervilliers 93,2.266712,G,Aubervilliers (93),93001,]50 000-100 000],Seine-Saint-Denis,Île-de-France,1.949633,"-0,137055837563452","2,10583333333333","1,97333333333333","1,915","1,625","2,129",200.0,68.0,"2,71",1.0,G
21,Bagnères-de-Bigorre 65,2.823087,E,Bagnères-de-Bigorre (65),65059,[0-20 000],Hautes-Pyrénées,Occitanie,2.278010,"-0,166666666666667","2,80845771144279","2,33830845771144","1,42686567164179","1,84328358208955","2,97313432835821",67.0,366.0,"2,76",1.0,G
40,Carcassonne 11,2.539395,F,Carcassonne (11),11069,[20 000-50 000],Aude,Occitanie,2.241303,"0,383720930232558","2,36973180076628","1,80459770114943","2,01149425287356","2,19540229885057","2,82528735632184",87.0,195.0,"2,68",1.0,G
41,Carrières-sous-Poissy 78,3.096306,E,Carrières-sous-Poissy (78),78123,[0-20 000],Yvelines,Île-de-France,2.168282,"-0,40625","2,52233676975945","2,21134020618557","2,2","2,04381443298969","1,8639175257732",97.0,392.0,"2,76",1.0,G
50,Châtillon 92,3.185313,D,Châtillon (92),92020,[20 000-50 000],Hauts-de-Seine,Île-de-France,2.271535,"-0,21031746031746","2,4245406824147","2,29199475065617","2,28661417322835","1,77263779527559","2,58188976377953",254.0,191.0,"2,68",1.0,G
60,Deuil-la-Barre 95,2.608378,F,Deuil-la-Barre (95),95197,[20 000-50 000],Val-d'Oise,Île-de-France,2.228512,"-0,160714285714286","2,61904761904762","2,3422619047619","1,85714285714286","2,06696428571429","2,25714285714286",56.0,199.0,"2,68",1.0,G
63,Eaubonne 95,2.952698,E,Eaubonne (95),95203,[20 000-50 000],Val-d'Oise,Île-de-France,2.215216,"-0,198473282442748","2,65903307888041","2,38422391857506","1,89770992366412","2,04961832061069","2,08549618320611",131.0,200.0,"2,68",1.0,G


In [75]:
df_note.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes.csv', index=False, sep=';')

In [76]:
notes_ville=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes.csv',  sep=';')

In [77]:
notes_ville[['Ville', 'moyenneIndicateur_global', 'lettre']]

,Ville,moyenneIndicateur_global,lettre
0,Acigné 35,4.726466,A+
1,Aix-en-Provence 13,3.253586,D
2,Aix-les-Bains 73,3.903217,B
3,Ajaccio 20,2.453236,F
4,Albi 81,3.486289,D
...,...,...,...
194,Vincennes 94,4.025573,B
195,Vitré 35,3.320939,D
196,Vitrolles 13,3.322451,D
197,Vitry-sur-Seine 94,2.673956,F


In [78]:
def valeur_taille(df1):
    liste=[]
    for i in df1['Taille_commune_2']:
        if i=='De 5000 et 19 999 habitants':
            liste.append('5 M-20 M habitants')
        elif i =='De 100 000 et 199 999 habitants':
            liste.append('100 M-200 M habitants')
        elif i =='De 20 000 et 49 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='De 50 000 et 99 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='200 000 habitants ou plus':
            liste.append('200 M habitants')
        elif i =='Moins de 5000 habitants':
            liste.append('-5 M habitants')
        else :
            liste.append(np.nan)
    df1['Taille']=np.array(liste)
valeur_taille(df)

In [79]:
df_note

,Ville,moyenneIndicateur_global,lettre,Nom_com_dep,Depcom,Classe,Dep_nom,Reg_nom,note_globale_moy,note_progres_moy,note_ressenti_general_moy,note_securite_moy,note_confort_moy,note_importance_velo_moy,note_stat_services_moy,n_tot_valide_net,classement_note_globale_classe,note_globale_moy_classe,Climat_Velo_num,lettre_cycl
0,Acigné 35,4.726466,A+,Acigné (35),35001,[0-20 000],Ille-et-Vilaine,Bretagne,3.575816,"0,112244897959184","3,8265306122449","3,40816326530612","3,53061224489796","3,75255102040816","3,36122448979592",98.0,25.0,"2,76",5.0,C
1,Aix-en-Provence 13,3.253586,D,Aix-en-Provence (13),13001,]100 000-200 000],Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,2.396763,"0,242693773824651","2,44760101010101","2,16835016835017","2,42070707070707","2,13352272727273","2,81363636363636",792.0,24.0,"2,79",2.0,F
2,Aix-les-Bains 73,3.903217,B,Aix-les-Bains (73),73008,[20 000-50 000],Savoie,Auvergne-Rhône-Alpes,3.324386,"0,389473684210526","3,25789473684211","2,96140350877193","3,52","3,08684210526316","3,79578947368421",95.0,14.0,"2,68",4.0,D
3,Ajaccio 20,2.453236,F,Ajaccio (2A),2A004,]50 000-100 000],Corse-du-Sud,Corse,2.039203,"-0,142857142857143","2,10392156862745","1,68169934640523","1,49098039215686","1,82450980392157","3,09490196078431",255.0,67.0,"2,71",1.0,G
4,Albi 81,3.486289,D,Albi (81),81004,[20 000-50 000],Tarn,Occitanie,3.156492,"0,325358851674641","3,28730158730159","2,7468253968254","3,28095238095238","3,07976190476191","3,38761904761905",210.0,29.0,"2,68",4.0,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,Vincennes 94,4.025573,B,Vincennes (94),94080,[20 000-50 000],Val-de-Marne,Île-de-France,3.230566,"0,5311004784689","3,30581761006289","3,0998427672956","3,35377358490566","3,12735849056604","3,26603773584906",212.0,25.0,"2,68",4.0,D
195,Vitré 35,3.320939,D,Vitré (35),35360,[0-20 000],Ille-et-Vilaine,Bretagne,2.710063,"0,0253164556962025","2,80922431865828","2,39832285115304","2,71194968553459","2,36163522012579","3,26918238993711",159.0,212.0,"2,76",3.0,E
196,Vitrolles 13,3.322451,D,Vitrolles (13),13117,[20 000-50 000],Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,2.561344,"0,557377049180328","2,74731182795699","2,38440860215054","2,56451612903226","2,74596774193548","2,36451612903226",62.0,135.0,"2,68",2.0,F
197,Vitry-sur-Seine 94,2.673956,F,Vitry-sur-Seine (94),94081,]50 000-100 000],Val-de-Marne,Île-de-France,2.063053,"-0,389473684210526","2,23859649122807","2,14561403508772","1,82947368421053","1,77105263157895","2,33052631578947",95.0,64.0,"2,71",1.0,G


In [85]:
#temp=pd.DataFrame()
#for ville in df_note['Ville'] :
#    temp.concat(temp, df[])
df_voulu=pd.DataFrame()
temp=df_note.merge(df[['Ville', 'Taille', 'Dep']], how='left')
#for ville in df['Ville'].unique():
    #pd.concat([df_note, temp[temp['Ville']==ville].loc(0)])
temp=temp[['Ville', 'moyenneIndicateur_global', 'lettre', 'Taille', 'Dep']]

In [97]:
temp.drop_duplicates(subset ="Ville",keep = 'first', inplace=True)

In [98]:
temp[temp['Ville']=='Grenoble 38']

,Ville,moyenneIndicateur_global,lettre,Taille,Dep
7801,Grenoble 38,3.564873,C,100 M-200 M habitants,38


In [104]:
temp['moyenne_globale']=(temp['moyenneIndicateur_global']-1)*4

In [105]:
temp=temp[['Ville', 'moyenne_globale', 'lettre', 'Taille', 'Dep']]

In [106]:
temp

,Ville,moyenne_globale,lettre,Taille,Dep
0,Acigné 35,14.905863,A+,5 M-20 M habitants,35
49,Aix-en-Provence 13,9.014344,D,100 M-200 M habitants,13
270,Aix-les-Bains 73,11.612869,B,20 M-100 M habitants,73
335,Ajaccio 20,5.812946,F,20 M-100 M habitants,2A
446,Albi 81,9.945157,D,20 M-100 M habitants,81
...,...,...,...,...,...
26293,Vincennes 94,12.102293,B,20 M-100 M habitants,94
26347,Vitré 35,9.283756,D,5 M-20 M habitants,35
26424,Vitrolles 13,9.289805,D,20 M-100 M habitants,13
26499,Vitry-sur-Seine 94,6.695823,F,20 M-100 M habitants,94


In [107]:
temp.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes_villes.csv', index=False, sep=';')

In [52]:
df2=df.copy()
df2['Indic20']=(df2['Indicateur_global']-1)*4
def tt_dataville_(df, var_note, commune='Ville') :
    liste_ville=df[commune].unique()
    grosse_ville=[]
    for ville in liste_ville :
        df_ville1=df[df[commune]==ville]
        if df_ville1[commune].count()>=50 :
            grosse_ville+=[ville]#on va se concentrer sur les grandes villes
    print(len(grosse_ville))
    dataville=pd.DataFrame()
    dataville[commune]=np.array(grosse_ville)
    for col in var_note:
        liste=[]
        liste_ecart=[]
        for ville in grosse_ville:
            donnees=df[df[commune]==ville]
            moyenne=np.mean(donnees[col])
            liste.append(moyenne)
            ecart=np.std(donnees[col])
            liste_ecart.append(ecart)
        dataville['moyenne'+col]=np.array(liste)
        dataville[col+'ecart']=np.array(liste_ecart)
    return dataville
dataville_indic=tt_dataville_(df2, var_note=['Indic20'])
dataville_indic

154


,Ville,moyenneIndic20,Indic20ecart
0,Aix-en-Provence 13,9.014344,3.525864
1,Aix-les-Bains 73,11.612869,4.047788
2,Ajaccio 20,5.812946,2.876086
3,Albi 81,9.945157,3.760408
4,Alfortville 94,5.716891,2.954646
...,...,...,...
149,Villeurbanne 69,8.922623,3.189311
150,Vincennes 94,12.102293,3.074060
151,Vitré 35,9.283756,3.266763
152,Vitrolles 13,9.289805,3.749808


In [55]:
dataville_indic.sort_values('Indic20ecart')

,Ville,moyenneIndic20,Indic20ecart
47,Cugnaux 31,8.531988,2.162562
13,Aubervilliers 93,5.066848,2.181169
83,Marseille 13,5.043468,2.530629
117,Romans-sur-Isère 26,8.365936,2.532971
63,La Haie-Fouassière 44,9.375249,2.595594
...,...,...,...
1,Aix-les-Bains 73,11.612869,4.047788
22,Bois-le-Roi 77,9.672767,4.079781
130,Salon-de-Provence 13,10.020762,4.156717
82,Marignane 13,10.604996,4.161785


In [38]:
#ville de 10 + np.mean(dataville_indic['Indicateur_globalecart']) 0.8043226613391314

0.8043226613391314

In [40]:
#ville de 2 + 0.6309006805467736
np.mean(dataville_indic['Indicateur_globalecart'])

0.6309006805467736

In [42]:
#ville de 50 + 0.8208584048938732
np.mean(dataville_indic['Indicateur_globalecart'])

0.8208584048938732

In [44]:
#ville de 30 + 0.8197525610319493
np.mean(dataville_indic['Indicateur_globalecart'])

0.8197525610319493

In [49]:
np.mean(dataville_indic['Indic20ecart'])

3.279010244127797

In [51]:
(3.25*1.96*2)**2

162.3076

In [56]:
(3.2*1.96)**2

39.337984000000006